In [1]:
#importamos la bibliotecas 
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt

In [3]:
#Esteblecermos el universo de las variables 

# Variables de Entrada
# ==========================
# luz exterior -> [0, 100]
# temperatura -> [0,100]
# ==========================

#Variable de salida 
# ==========================
# potenciaFoco -> [0, 100]  
# ==========================


#con respecto a la salida del tipo sugeno podemos tener en 
# cuenta que los valores serian  
# potenciaFoco ->[0, 50, 100]
# y en mandami 
# potenciaFoco -> [Apagado - medioEncendido - Encendido]
#Apagado        = [-273, 0.566, 75]
#medioEncendido = [50, 135, 200]
#Encendido      = [200, 250, 250]

In [4]:
x_luz =np.arange(0, 100, 1)
x_temperatura =np.arange(0, 100, 1)
#===================================================================
#en este caso dado que la intencidad sera controlada por medio de un 
#actuador se transforma la salida a conrde al sistema binaria
#de la misma forma que se haria con un arduino para el control de luces led
#===================================================================
x_potenciaFoco =np.arange(0, 255, 1)

In [ ]:
#ahora vamos a generar las funcinoes de membresia difusa
# Variables de Entrada
# ==========================
# luz exterior -> [baja , ambiente , alta]
#recordemos que en esta variable tenemos del tipo trapezoidal y triangular
# ==========================
 LB = fuzz.trapmf(x_luz,[-30.24, -9.76, 9.76, 30.24])
LAB = fuzz.trimf(x_luz,[10, 50, 90])
 LA = fuzz.trapmf(x_luz,[70, 85, 100, 100])

# temperatura -> [frio , ambiente , caliente]
# ==========================